In [16]:
import numpy as np
import matplotlib.pyplot as plt
import os
from enum import Enum
from itertools import permutations, combinations, product
from copy import copy
from icecream import ic
from math import atan
import logging
from functools import cmp_to_key
from tabulate import tabulate

### Input format
The first line is of the form
```
num_agents est_o dl_o
```

It is followed by `num_agents` lines, of which line $i$ (zero-indexed) of the input .csv file pertains to issue $i$, and is a tuple of the form `(agent, parent_issue, [pretasks], children_finish, issue_type, other_issue, pbs, reward, decommitment_penalty_rate)`, where

- `agent` is the agent ID of that agent which is supposed to do this task
- `parent_issue` of an incoming issue is **not** the same as that of its `other_issue`
- `[pretasks]` = Set of explicit pretasks (i.e., doesn't include parent-child relationships). '' if there is no pretask. Separated by ';' otherwise
- `children_finish` = 0 for "and", 1 for "or", '' for no chlidren. It may be 0 or 1 even if it has no children; however, '' indicates that we know for sure that there is no child.
- `issue_type` = 1 for local tasks (that are not incoming), 2 for outgoing tasks, 3 for incoming tasks
- `other_issue` is the issue ID of the issue linked to this issue (defined only when issue_type is outgoing or incoming)
- `pbs` is the base probability of success
- `reward` is the completion reward of the task, defined only for incoming issues
- `decommitment_penalty_rate` is the decommitment penalty rate, defined only for incoming issues

In [17]:
class issue_type(Enum):
    NONE = 0
    LOCAL = 1
    OUTGOING = 2
    INCOMING = 3


class issue:
    def __init__(self, issue_num: int, line: str) -> None:
        self.ID = issue_num
        line = line.split(",")
        self.agent = int(line[0])
        self.parent = int(line[1])
        try:
            self.pretasks = list(map(int, line[2].split(";")))
        except:
            self.pretasks = []

        self.children_finish = int(line[3])
        self.issue_type = issue_type(int(line[4]))
        self.other_issue = int(line[5])
        self.pbs = float(line[6])
        if self.issue_type == issue_type.INCOMING:
            self.reward = float(line[7])
            self.decom_penalty_rate = float(line[8])
        else:
            self.reward = 0
            self.decom_penalty_rate = 0

    def __str__(self) -> str:
        return f"agent:{self.agent} parent:{self.parent} pretasks:{self.pretasks} children_finish:{self.children_finish} issue_type:{self.issue_type.name} other_issue:{self.other_issue} pbs:{self.pbs} r:{self.reward} p:{self.decom_penalty_rate}"


class agent:
    def __init__(self) -> None:
        self.issues: list[int] = []
        # issues that are either outgoing or incoming
        self.negotiation_issues: list[int] = []

In [18]:
input_file = os.path.join(os.getcwd(), "..", "data", "input.csv")

issue_graph: list[issue] = []
with open(input_file, "r") as f:
    num_agents, est_o, dl_o = list(map(int, f.readline().split()))
    agents: list[agent] = [agent() for i in range(num_agents)]
    issue_num: int = 0
    for line in f:
        cur_issue = issue(issue_num, line)
        issue_graph.append(cur_issue)
        agents[cur_issue.agent].issues.append(issue_num)
        issue_num += 1

print(f"{est_o=}")
print(f"{dl_o=}")
print(f"{num_agents=}")
print("Agents and their issues:")
for i, agent in enumerate(agents):
    print(f"Agent {i}: {agent.issues}")

print("\nIssues and their components:")
# TODO: tabulate this instead
print(
    tabulate(
        [
            [
                i,
                issue_graph[i].agent,
                issue_graph[i].parent,
                issue_graph[i].pretasks,
                issue_graph[i].children_finish,
                issue_graph[i].issue_type.name,
                issue_graph[i].other_issue,
                issue_graph[i].pbs,
                issue_graph[i].reward,
                issue_graph[i].decom_penalty_rate,
            ]
            for i in range(len(issue_graph))
        ],
        headers=[
            "ID",
            "agent",
            "parent",
            "pretasks",
            "children_finish",
            "issue_type",
            "other_issue",
            "pbs",
            "reward",
            "decom_penalty_rate",
        ],
    )
)

est_o=0
dl_o=10
num_agents=3
Agents and their issues:
Agent 0: [0, 1]
Agent 1: [2, 3, 4, 5, 6, 7, 8]
Agent 2: [9, 10, 11, 12]

Issues and their components:
  ID    agent    parent  pretasks      children_finish  issue_type      other_issue    pbs    reward    decom_penalty_rate
----  -------  --------  ----------  -----------------  ------------  -------------  -----  --------  --------------------
   0        0        -1  []                          0  OUTGOING                  2   0.98         0                   0
   1        0        -1  []                          0  OUTGOING                 10   0.89         0                   0
   2        1        -1  []                          0  INCOMING                  0   0.7         10                   1
   3        1         2  []                          0  LOCAL                    -1   0.68         0                   0
   4        1         3  []                          0  LOCAL                    -1   1            0              

Finding those issues that are going to be negotiated over:

In [19]:
print("Negotiation issues:")
for i, agent in enumerate(agents):
    agent.negotiation_issues = [
        issueID
        for issueID in agent.issues
        if issue_graph[issueID].issue_type in [issue_type.OUTGOING, issue_type.INCOMING]
    ]
    agent.negotiation_issues.sort()
    print(f"Agent {i}: {agent.negotiation_issues}")

Negotiation issues:
Agent 0: [0, 1]
Agent 1: [2, 6]
Agent 2: [9, 10]


### Complete search for the optimal (negotiation ordering, feature assignment tuple)

Helper functions to calculate flexibility, ps (probability of success), and EV (expected value):

In [20]:
def flexibility(est, dl, duration) -> float:
    return (dl - est - duration) / duration


def ps(pbs, est, dl, duration, c=0):
    return 2 * pbs * (atan(flexibility(est, dl, duration))) / np.pi


def EV(
    # TODO: remove ordering from args below?
    # ordering: tuple,
    assignment: tuple[tuple[int, int, int]],
    agent_ID: int,
) -> float:
    ev: float = 0
    f = [flexibility(est, dl, dur) for (est, dl, dur) in assignment]
    pbs_list = [
        issue_graph[issueID].pbs for issueID in agents[agent_ID].negotiation_issues
    ]
    ps_list = [
        ps(pbs, est, dl, dur) for (est, dl, dur), pbs in zip(assignment, pbs_list)
    ]
    issue_ID_list = [issueID for issueID in agents[agent_ID].negotiation_issues]
    for outcome in product([0, 1], repeat=len(assignment)):
        p = np.prod([ps if outcome[i] else 1 - ps for i, ps in enumerate(ps_list)])
        r = np.prod(
            [
                issue_graph[issueID].reward
                if outcome[i]
                else issue_graph[issueID].reward
                * issue_graph[issueID].decom_penalty_rate
                for i, issueID in zip(range(len(outcome)), issue_ID_list)
            ]
        )
        ev += p * r
    return ev


def is_valid_assignment(assignment) -> bool:
    for (est, dl, dur) in assignment:
        if est + dur > dl or est >= dl:
            return False

    def compare(a, b):
        # TODO: this may not be correct, how to find out if an assignment is valid?
        if a[0] != b[0]:
            return a[1] - b[1]  # deadline
        else:
            return a[0] - b[0]  # est

    sorted_ass = list(assignment)
    sorted_ass.sort(key=cmp_to_key(compare))
    # print(f'{sorted_ass=}')
    t = -100  # any negative number suffices since est >= 0 so t+1 >= 0
    for (est, dl, dur) in sorted_ass:
        t = max(t + 1, est) + dur
        if t > min(dl, dl_o):
            return False

    return True

The complete search itself:

In [23]:
for agent_ID, agent in enumerate(agents):
    best_value = float("-inf")
    best_ordering = None
    best_assignment = None
    for ordering in permutations(agent.negotiation_issues):
        est_space = range(est_o, dl_o)
        dl_space = range(est_o, dl_o + 1)
        duration_space = range(1, dl_o - est_o)
        for assignment in product(
            product(est_space, dl_space, duration_space), repeat=len(ordering)
        ):
            # print(f"Ordering: {ordering}")
            # print(f"Assignment: {assignment}")
            if is_valid_assignment(assignment):
                ev = EV(assignment, agent_ID)
                if ev > best_value:
                    best_value = ev
                    best_ordering = ordering
                    best_assignment = assignment
    print(f'Agent: {agent_ID}:')
    print(f'Best value = {best_value}')
    print(f'Best ordering (this may be redundant or even wrong (but irrelevant)): {ordering}')
    print(f'Best assignment: {best_assignment}\n')

Agent: 0:
Best value = 0.0
Best ordering (this may be redundant or even wrong (but irrelevant)): (1, 0)
Best assignment: ((0, 1, 1), (0, 3, 1))
Agent: 1:
Best value = 0.0
Best ordering (this may be redundant or even wrong (but irrelevant)): (6, 2)
Best assignment: ((0, 1, 1), (0, 3, 1))
Agent: 2:
Best value = 165.14551638756683
Best ordering (this may be redundant or even wrong (but irrelevant)): (10, 9)
Best assignment: ((0, 1, 1), (0, 10, 1))
